In [41]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime,date
import glob
from pprint import pprint

In [2]:
# parentDir = os.path.abspath(os.path.join(os.getcwd(),'..'))
# os.chdir(parentDir)

# Check data_loader

## Check load_transaction

In [43]:


def load_transaction(data_folder_path, transaction_file_pattern):
    transaction_files = _gather_transaction_files(data_folder_path,transaction_file_pattern)
    transactions = _combine_transaction_files(transaction_files)
    transactions = _clean_transactions(transactions)
    print(f"The latest transaction date is {transactions['Run Date'].max()}")
    return transactions


def _gather_transaction_files(data_folder_path,transaction_file_pattern):
    transaction_file_path_pattern = os.path.join(
        data_folder_path, transaction_file_pattern
    )
    transaction_files = glob.glob(transaction_file_path_pattern)
    return transaction_files

def _combine_transaction_files(transaction_files):
    transaction_list = [
        pd.read_csv(file, usecols=range(14)) for file in transaction_files
    ]
    transactions = pd.concat(transaction_list, ignore_index=True)
    return transactions

def _remove_NA_value(df,colName):
    df_copy = df.copy()
    df_copy = df_copy[
        df_copy[colName].notna()
    ] 
    return df_copy

def _remove_leading_space(df,colName):
    df_copy = df.copy()
    df_copy[colName] = df_copy[colName].str.lstrip()
    return df_copy

def _str_to_date(df, colName, format):
    df_copy = df.copy()
    df_copy[colName] = pd.to_datetime(
        df_copy[colName], format=format
    ).dt.date
    return df_copy

def _add_Transfer_symbol(df):
    df_copy = df.copy()
    df_copy.loc[df_copy["Symbol"] == "  ", "Symbol"] = "Transfer"
    return df_copy

def _sort_df_by_column(df, colName):
    df_copy = df.copy()
    df_copy = df_copy.sort_values(by=colName).reset_index(
        drop=True
    )
    return df_copy


def _clean_transactions(transactions):
    transactions = _remove_NA_value(transactions,"Amount ($)")
    transactions = _remove_leading_space(transactions,"Run Date")
    transactions = _str_to_date(transactions,"Run Date","%m/%d/%Y")
    transactions = _str_to_date(transactions,"Settlement Date","%m/%d/%Y")
    transactions = _add_Transfer_symbol(transactions)
    transactions = _remove_leading_space(transactions,"Symbol")
    transactions = _remove_leading_space(transactions,"Description")
    transactions = _sort_df_by_column(transactions,"Run Date")
    return transactions


In [44]:
data_folder_path = "./data"
transaction_file_pattern = "Accounts_History_*.csv"
position_file_pattern = "Portfolio_Positions_*.csv"

In [45]:
transaction_files = _gather_transaction_files(data_folder_path,transaction_file_pattern)
transactions = _combine_transaction_files(transaction_files)
transactions = _clean_transactions(transactions)

In [46]:
# transactions = _remove_NA_value(transactions,"Amount ($)")
# transactions = _remove_leading_space(transactions,"Run Date")
# transactions = _str_to_date(transactions,"Run Date","%m/%d/%Y")
# transactions = _str_to_date(transactions,"Settlement Date","%m/%d/%Y")
# transactions = _add_Transfer_symbol(transactions)
# transactions = _remove_leading_space(transactions,"Symbol")
# transactions = _sort_df_by_column(transactions,"Run Date")

## Check load_position

In [47]:

def load_position(data_folder_path, position_file_pattern):
    position_file = _gather_position_files(data_folder_path, position_file_pattern)

    if not position_file is None:
        position = pd.read_csv(position_file)
        position = _clean_position(position)
    else:
        print("No position file found.")
    return position

def _gather_position_files(data_folder_path, position_file_pattern):
    position_file_path_pattern = os.path.join(data_folder_path, position_file_pattern)
    position_files = glob.glob(position_file_path_pattern)
    position_file = _find_latest_position_file(position_files)
    return position_file


def _find_latest_position_file(position_files):
    latest_file = None
    latest_date = None

    for file_path in position_files:
        file_name = os.path.basename(file_path)
        date_str = file_name.split("_")[-1].replace(".csv", "")
        file_date = datetime.strptime(date_str, "%b-%d-%Y")

        if latest_date is None or file_date > latest_date:
            latest_date = file_date
            latest_file = file_path

    return latest_file


def _clean_position(position):
    position = _remove_NA_value(position,"Current Value")
    position = _transfer_dollar_to_float(position, "Current Value")
    position = _transfer_dollar_to_float(position, "Cost Basis Total")
    return position


def _transfer_dollar_to_float(df, colNames):
    """
    Change "$123,456" to 123456.0, and "--" to 0.0
    """
    df_copy = df.copy()
    # Replace any "--" with "$0"
    cleaned = df_copy[colNames].str.replace("--", "$0", regex=False)
    # Remove dollar sign and commas, then convert to float
    cleaned = cleaned.str.replace("$", "", regex=False).str.replace(",", "", regex=False)
    df_copy[colNames] = cleaned.astype(float)
    return df_copy


In [48]:
data_folder_path = "./data"
transaction_file_pattern = "Accounts_History_*.csv"
position_file_pattern = "Portfolio_Positions_*.csv"

In [49]:
position_file = _gather_position_files(data_folder_path, position_file_pattern)
position_file

'./data/Portfolio_Positions_Aug-05-2025.csv'

In [50]:
position = pd.read_csv(position_file)
position = _clean_position(position)

In [51]:
# position = _remove_NA_value(position,"Current Value")
# position = _transfer_dollar_to_float(position, "Current Value")
# position = _transfer_dollar_to_float(position, "Cost Basis Total")

# Check Portfolio

In [52]:
from scipy.optimize import newton
import numpy as np
def compute_irr(cashflows, dates, cob):
    """用 Newton-Raphson 方法计算 IRR（连续复利）"""
    def npv(r):
        return sum(cf * np.exp(-r * (cob - d).days / 365.0) for cf, d in zip(cashflows, dates))
    try:
        result = -newton(npv, 0.1)
    except RuntimeError:
        result = np.nan
    return result

def display_percentage(df, colNames: list):
    df_copy = df.copy()
    for col in colNames:
        df_copy[col] = df_copy[col].apply(lambda x: f"{x:.2%}")
    return df_copy

In [ ]:
class Portfolio:
    def __init__(self, transactions, position):
        self.transactions = transactions
        self.position = position
        self.cob = date.today()
        
        self.account_number_dic = {
            "Individual":'Z23390746',
            "401k":'86964',
            "HSA":'241802439',
            "Cash":'Z06872898'
        }
        
        self.pensionTransactions = transactions[transactions['Account Number']==self.account_number_dic["401k"]]
        self.pensionPosition = position[position['Account Number']==self.account_number_dic["401k"]]
        self.HSATransactions = transactions[transactions['Account Number']==self.account_number_dic["HSA"]]
        self.HSAPosition = position[position['Account Number']==self.account_number_dic["HSA"]]
        self.cashTransactions = transactions[transactions['Account Number']==self.account_number_dic["Cash"]]
        self.cashPosition = position[position['Account Number']==self.account_number_dic["Cash"]]
        
        
        self.cashSymbols = ['FZFXX**','FZFXX']
        self.otherSymbols = ['Pending Activity']
        self.contributionSymbols = ['','Transfer']
        self.bondSymbols = self.get_bond_symbol_list()
        self.stockSymbols = self.get_stock_symbol_list()
        
        self.cashDescriptions = ['HELD IN MONEY MARKET','FIDELITY TREASURY MONEY MARKET FUND']
        self.otherDescriptions = []
        self.contributionDescriptions = ['No Description']
        self.bondDescriptions = self.get_bond_description_list()
        self.stockDescriptions = self.get_stock_description_list()
        
    ## Main functions
    
    def get_account_summary(self):
        key = 'Symbol'
        if key == 'Symbol':
            listKeys = self.get_bond_symbol_list()
        elif key == ' Description':
            listKeys = self.get_bond_description_list()
        else:
            return 0
        bondTotalValue = self.get_total_keys_value(listKeys,key)
        bondTotalIrr = self.get_combined_key_irr(listKeys,key)
        bondHoldingPeriod = self.get_combined_symbol_holding_period(listKeys,key)
        return bondTotalValue, bondTotalIrr, bondHoldingPeriod
        
        stockSymbols = self.get_stock_symbol_list()
        stockTotalValue = self.get_total_keys_value(stockSymbols,key)
        stockTotalIrr = self.get_combined_key_irr(stockSymbols,key)
        stockHoldingPeriod = self.get_combined_key_holding_period(stockSymbols,key)
        
        cashSymbols = self.cashSymbols
        cashTotalValue = self.get_total_keys_value(cashSymbols,key)
        cashTotalIrr = self.get_combined_key_irr(cashSymbols,key)
        cashHoldingPeriod = self.get_combined_key_holding_period(cashSymbols,key)
        
        totalIRR = self.get_total_irr()
        totalHoldingPeriod = self.get_combined_symbol_holding_period(cashSymbols+stockSymbols+bondSymbols)
        
        totalValue = self.Position['Current Value'].sum()
        result = pd.DataFrame({
            'Type':['bond','stock','cash','Total'],
            'Value': [bondTotalValue,stockTotalValue,cashTotalValue,totalValue],
            'Percentage':[bondTotalValue,stockTotalValue,cashTotalValue,totalValue]/totalValue,
            'IRR':[bondTotalIrr, stockTotalIrr, cashTotalIrr,totalIRR],
            'Weighted Avg Holding Period':[bondHoldingPeriod,stockHoldingPeriod,cashHoldingPeriod,totalHoldingPeriod]
        } 
        )
        return result
    
        
    def get_all_stock_summary(self, colName='Description'):
        stockSymbols = self.get_stock_symbol_list()
        currentValueResult = self.get_key_current_values(stockSymbols,colName)
        irrResult = self.get_key_irrs(stockSymbols,colName)
        holdingPeriodResult = self.get_symbol_holding_period(stockSymbols,colName)
        result = pd.merge(currentValueResult, irrResult, on=colName)
        result = pd.merge(result, holdingPeriodResult, on=colName)
        result = result.sort_values(by='Current Value', ascending=False)
        return result
    
    def get_all_bond_summary(self,showExpiredBond=False):
        bondSymbols = self.get_bond_symbol_list()
        currentValueResult = self.get_key_current_values(bondSymbols)
        irrResult = self.get_key_irrs(bondSymbols)
        holdingPeriodResult = self.get_symbol_holding_period(bondSymbols)
        result = pd.merge(currentValueResult, irrResult, on='Symbol')
        result = pd.merge(result, holdingPeriodResult, on='Symbol')
        result = result.sort_values(by='Current Value', ascending=False)
        if not showExpiredBond:
            result = result[result['Current Value']>0]
        return result
    
    
    ## Help functions
    
    def get_combined_bond_result(self, colName):
        if colName == 'Symbol':
            listKeys = self.get_bond_symbol_list()
        elif colName == 'Description':
            listKeys = self.get_bond_description_list()
        else:
            return 0
        totalValue = self.get_total_keys_value(listKeys,colName)
        totalIrr = self.get_combined_key_irr(listKeys,colName)
        holdingPeriod = self.get_combined_key_holding_period(listKeys,colName)
        return totalValue, totalIrr, holdingPeriod
    
    def get_combined_stock_result(self, colName):
        if colName == 'Symbol':
            listKeys = self.get_stock_symbol_list()
        elif colName == 'Description':
            listKeys = self.get_stock_description_list()
        else:
            return 0
        totalValue = self.get_total_keys_value(listKeys,colName)
        totalIrr = self.get_combined_key_irr(listKeys,colName)
        holdingPeriod = self.get_combined_key_holding_period(listKeys,colName)
        return totalValue, totalIrr, holdingPeriod
    
    def get_combined_key_holding_period(self, listKeys: list, colName, unit = 30):
        subTransactions = self.transactions[self.transactions[colName].isin(listKeys)]
        buyTranactions = subTransactions[subTransactions['Amount ($)']<0]
        df = buyTranactions.copy()
        df['Days Held'] = (self.cob - df['Run Date']).apply(lambda x: x.days)
        df['Weight'] = df['Amount ($)'].abs()
        totalWeightedHold = (df['Days Held'] * df['Weight']).sum() / df['Weight'].sum()/ unit
        return totalWeightedHold
        
    def get_key_holding_period(self, listKeys: list, colName, unit = 30):
        subTransactions = self.transactions[self.transactions[colName].isin(listKeys)]
        buyTranactions = subTransactions[subTransactions['Amount ($)']<0]
        df = buyTranactions.copy()
        df['Days Held'] = (self.cob - df['Run Date']).apply(lambda x: x.days)
        df['Weight'] = df['Amount ($)'].abs()
        totalWeightedHold = (df['Days Held'] * df['Weight']).sum() / df['Weight'].sum()/ unit
        totalWeightedHoldRow = pd.DataFrame({'colName': ['Total'], 'Weighted Avg Holding Period': [totalWeightedHold]})

        # 分组计算加权平均
        weightedHold = (
            df.groupby('Symbol')
            .apply(lambda g: (g['Days Held'] * g['Weight']).sum() / g['Weight'].sum()/ unit, include_groups=False)
            .reset_index(name='Weighted Avg Holding Period')
        )
        
        weightedHold = pd.concat([weightedHold, totalWeightedHoldRow], ignore_index=True)
        return weightedHold
    
    
    def get_key_current_values(self, listKeys: list, colName):
        resultList = []
        totalCurrentValue = self.get_total_keys_value(listKeys, colName)
        for key in listKeys:
            currentValue = self.get_key_current_value(key, colName)
            currentValuePercent = currentValue/totalCurrentValue
            resultList.append({
                colName: key,
                'Current Value': currentValue,
                'Percentage': currentValuePercent
            })
        resultList.append({
                colName: 'Total',
                'Current Value': totalCurrentValue,
                'Percentage': 1
            })
        return pd.DataFrame(resultList)
    
    def get_key_current_value(self,key, colName):
        try:
            value = self.position.loc[self.position[colName]==key, 'Current Value'].values[0]
        except:
            value = 0
        return value
        
    def get_total_keys_value(self, listKeys: list, colName):
        Position = self.position[self.position[colName].isin(listKeys)]
        totalValue = Position['Current Value'].sum()
        return totalValue
    
    def get_key_irrs(self, listKeys: list, colName):
        resultList = []
        for key in listKeys:
            irr = self.get_combined_key_irr([key], colName)
            resultList.append({
                colName: key,
                'IRR': irr
            })
        totalIrr = self.get_combined_key_irr(listKeys, colName)
        resultList.append({
                colName: 'Total',
                'IRR': totalIrr
            })
        return pd.DataFrame(resultList)
    
    def get_total_irr(self):
        trans = self.transactions[self.transactions['Symbol'].isin(self.contributionSymbols)]
        cashflows = trans['Amount ($)'].tolist()
        cashflows = [-x for x in cashflows]
        dates = trans['Run Date'].tolist()
        current_value = self.position['Current Value'].sum()
        cashflows.append(current_value)
        dates.append(self.cob)
        irr = compute_irr(cashflows, dates, self.cob)
        return irr
    
    
    def get_combined_key_irr(self, listKeys: list, colName):
        trans = self.transactions[self.transactions[colName].isin(listKeys)]
        cashflows = trans['Amount ($)'].tolist()
        dates = trans['Run Date'].tolist()
        current_value = self.position.loc[self.position[colName].isin(listKeys), 'Current Value'].sum()
        cashflows.append(current_value)
        dates.append(self.cob)
        irr = compute_irr(cashflows, dates, self.cob)
        return irr
    
    
    
    def get_stock_symbol_list(self):
        symbols = self.transactions['Symbol'].unique()
        stockSymbols = [
            sym for sym in symbols
            if sym not in self.cashSymbols
            and not str(sym).startswith('91')
            and sym not in self.otherSymbols
            and sym not in self.contributionSymbols
        ]
        return stockSymbols
    
    def get_stock_description_list(self):
        descriptions = self.transactions['Description'].unique()
        stockDescriptions = [
            desc for desc in descriptions
            if desc not in self.cashDescriptions
            and not str(desc).startswith('UNITED STATES TREAS BILLS')
            and desc not in self.otherDescriptions
            and desc not in self.contributionDescriptions
        ]
        return stockDescriptions
    
    def get_bond_symbol_list(self):
        symbols = self.transactions['Symbol'].unique()
        bondSymbols = [
            sym for sym in symbols
            if  str(sym).startswith('91')
        ]
        return bondSymbols
    
    def get_bond_description_list(self):
        descriptions = self.transactions['Description'].unique()
        bondDescriptions = [
            desc for desc in descriptions
            if  str(desc).startswith('UNITED STATES TREAS BILLS')
        ]
        return bondDescriptions
    
    
    def get_401k_description_list(self):
        descriptions = self.pensionTransactions['Description'].unique()
        return descriptions
    


class IndividualPortfolio(Portfolio):
    def __init__(self, transactions, position):
        super().__init__(transactions, position)
        self.transactions = transactions[transactions['Account Number']==self.account_number_dic["Individual"]]
        self.position = position[position['Account Number']==self.account_number_dic["Individual"]]
        

## Check get_individual_account_summary

### Fun Run

In [175]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
portfolio = IndividualPortfolio(transactions, position)

The latest transaction date is 2025-08-05


AttributeError: 'float' object has no attribute 'startswith'

In [172]:
symbols = transactions['Symbol'].unique()
bondSymbols = [
            sym for sym in symbols
            if  str(sym).startswith('91')
        ]

In [127]:
result = portfolio.get_individual_account_summary()
display_percentage(result,['Percentage','IRR'])

AttributeError: 'Portfolio' object has no attribute 'get_total_symbols_value'

### Step Run

In [152]:
colName = 'Description'
portfolio.get_combined_stock_result(colName)

(np.float64(418641.33999999997),
 np.float64(0.15238469164371765),
 np.float64(12.16879226493835))

In [153]:
colName = 'Symbol'
portfolio.get_combined_stock_result(colName)

(np.float64(423303.14),
 np.float64(0.16276691355116052),
 np.float64(12.16879226493835))

In [159]:
colName = 'Symbol'
if colName == 'Symbol':
    listKeys = portfolio.get_stock_symbol_list()
elif colName == 'Description':
    listKeys = portfolio.get_stock_description_list()
else:
    0
trans = portfolio.transactions[portfolio.transactions[colName].isin(listKeys)]
cashflows = trans['Amount ($)'].tolist()
dates = trans['Run Date'].tolist()
current_value = portfolio.position.loc[portfolio.position[colName].isin(listKeys), 'Current Value'].sum()
# cashflows.append(current_value)
# dates.append(portfolio.cob)

In [160]:
colName = 'Description'
if colName == 'Symbol':
    listKeys = portfolio.get_stock_symbol_list()
elif colName == 'Description':
    listKeys = portfolio.get_stock_description_list()
else:
    0
trans1 = portfolio.transactions[portfolio.transactions[colName].isin(listKeys)]
cashflows1 = trans1['Amount ($)'].tolist()
dates1 = trans1['Run Date'].tolist()
current_value1 = portfolio.position.loc[portfolio.position[colName].isin(listKeys), 'Current Value'].sum()
# cashflows.append(current_value)
# dates.append(portfolio.cob)

In [163]:
portfolio.position[portfolio.position[colName].isin(listKeys)]

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
0,Z23390746,Individual,FXAIX,FIDELITY 500 INDEX FUND,744.909,$219.98,+$3.20,163865.08,$0.00,0.00%,+$22343.33,+15.78%,13.71%,141521.75,$189.99,Cash
3,Z23390746,Individual,FSKAX,FIDELITY TOTAL MARKET INDEX FUND,651.878,$173.93,+$2.62,113381.14,$0.00,0.00%,+$12245.50,+12.10%,9.49%,101135.64,$155.15,Cash
8,Z23390746,Individual,FSPSX,FIDELITY INTERNATL INDEX FUND,1226.793,$56.65,+$0.72,69497.82,$0.00,0.00%,+$7855.18,+12.74%,5.81%,61642.64,$50.25,Cash
11,Z23390746,Individual,MSFT,MICROSOFT CORP,30.000,$528.90,-$6.74,15867.00,-$202.20,-1.26%,+$2524.05,+18.91%,1.33%,13342.95,$444.77,Cash
12,Z23390746,Individual,AAPL,APPLE INC,70.000,$203.3353,-$0.0147,14233.47,-$1.03,-0.01%,+$3473.48,+32.28%,1.19%,10759.99,$153.71,Cash
13,Z23390746,Individual,TSLA,TESLA INC COM,20.000,$309.67,+$0.41,6193.40,+$8.20,+0.13%,+$2425.90,+64.39%,0.52%,3767.50,$188.38,Cash
14,Z23390746,Individual,MCD,MCDONALD S CORP,20.000,$300.68,-$3.55,6013.60,-$71.00,-1.17%,+$856.10,+16.59%,0.50%,5157.50,$257.88,Cash
16,Z23390746,Individual,GOOGL,ALPHABET INC CAP STK CL A,20.000,$194.405,-$0.635,3888.10,-$12.70,-0.33%,+$2058.10,+112.46%,0.33%,1830.00,$91.50,Cash
17,Z23390746,Individual,AXP,AMERICAN EXPRESS CO COM USD0.20,10.000,$298.12,-$0.47,2981.20,-$4.70,-0.16%,+$1481.20,+98.74%,0.25%,1500.00,$150.00,Cash
18,Z23390746,Individual,JPM,JPMORGAN CHASE &CO. COM,10.000,$291.5429,-$2.7171,2915.42,-$27.18,-0.93%,+$1715.42,+142.95%,0.24%,1200.00,$120.00,Cash


In [ ]:

cashflows = trans['Amount ($)'].tolist()
dates = trans['Run Date'].tolist()
current_value = self.position.loc[self.position[colName].isin(listKeys), 'Current Value'].sum()
cashflows.append(current_value)
dates.append(self.cob)
irr = compute_irr(cashflows, dates, self.cob)

In [148]:
portfolio.get_total_keys_value(listKeys,colName)

np.float64(418641.33999999997)

In [149]:
portfolio.get_combined_key_irr(listKeys,colName)

nan

In [ ]:
trans = self.transactions[self.transactions['Symbol'].isin(listKeys)]

In [ ]:
trans = self.transactions[self.transactions['Symbol'].isin(listKeys)]
        cashflows = trans['Amount ($)'].tolist()
        dates = trans['Run Date'].tolist()
        current_value = self.position.loc[self.position['Symbol'].isin(listKeys), 'Current Value'].sum()
        cashflows.append(current_value)
        dates.append(self.cob)
        irr = compute_irr(cashflows, dates, self.cob)

## Check get_all_stock_summary

### Full Run

In [134]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


In [107]:
result = portfolio.get_all_stock_summary()
display_percentage(result,['Percentage','IRR']).head(5)

KeyError: 'Symbol'

### Step Run

#### Current Value

(np.float64(423303.14),
 np.float64(0.16276691355116052),
 np.float64(12.16879226493835))

In [120]:
stockSymbols = portfolio.get_stock_symbol_list()
currentValueResult = portfolio.get_key_current_values(stockSymbols,'Symbol').head(5)
currentValueResult.head(5)

,Key,Current Value,Percentage
0,AAPL,14233.47,0.033625
1,SBUX,2707.05,0.006395
2,JPM,2915.42,0.006887
3,AXP,2981.20,0.007043
4,GOOGL,3888.10,0.009185


In [121]:
stockDescriptions = portfolio.get_stock_description_list()
portfolio.get_key_current_values(stockDescriptions,'Description').head(5)

,Key,Current Value,Percentage
0,APPLE INC,14233.47,0.033999
1,STARBUCKS CORP COM USD0.001,2707.05,0.006466
2,JPMORGAN CHASE &CO. COM,2915.42,0.006964
3,AMERICAN EXPRESS CO COM USD0.20,2981.20,0.007121
4,ALPHABET INC CAP STK CL A,3888.10,0.009287


#### IRR

In [ ]:
irrResult = portfolio.get_key_irrs(stockSymbols, 'Symbol')
irrResult.head(5)

,Key,IRR
0,AAPL,0.102383
1,SBUX,0.065357
2,JPM,0.323283
3,AXP,0.245696
4,GOOGL,0.309569
5,AMZN,0.329863
6,NKE,-0.071231
7,TSLA,0.326300
8,FXAIX,0.195717
9,MCD,0.148729


In [89]:
holdingPeriod = portfolio.get_symbol_holding_period(stockSymbols)
holdingPeriod.head(5)

,Symbol,Weighted Avg Holding Period
0,AAPL,34.997598
1,AMZN,29.900000
2,AXP,35.666667
3,BRKB,9.600000
4,COKE,13.218008


,Key,Current Value,Percentage
0,APPLE INC,14233.47,0.033999
1,STARBUCKS CORP COM USD0.001,2707.05,0.006466
2,JPMORGAN CHASE &CO. COM,2915.42,0.006964
3,AMERICAN EXPRESS CO COM USD0.20,2981.20,0.007121
4,ALPHABET INC CAP STK CL A,3888.10,0.009287


In [ ]:
listKeys = stockDescriptions
colName = 'Description'

In [ ]:
resultList = []
totalCurrentValue = portfolio.get_total_keys_value(listKeys, colName)
totalCurrentValue

np.float64(418641.33999999997)

In [ ]:
key = listKeys[0]
print(key)
currentValue = portfolio.get_key_current_value(key)
currentValue

APPLE INC


0

## Check get_bond_irr

In [63]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


In [64]:
result = portfolio.get_all_bond_summary(showExpiredBond=False)
display_percentage(result,['Percentage','IRR'])

,Symbol,Current Value,Percentage,IRR,Weighted Avg Holding Period
54,Total,746743.5,100.00%,4.37%,6.113349
46,912797PW1,149338.5,20.00%,3.71%,0.820074
50,912797QU4,149253.0,19.99%,3.37%,0.899616
48,912797MG9,99976.0,13.39%,4.18%,1.433333
49,912797QL4,99751.0,13.36%,3.88%,1.366667
51,912797QT7,99586.0,13.34%,3.41%,0.900000
53,912797QW0,99336.0,13.30%,0.20%,0.200000
52,912797NA1,49503.0,6.63%,3.14%,0.300000


## Check get 401k account summary

In [77]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


AttributeError: 'Portfolio' object has no attribute 'get_bond_Describtions_list'

In [66]:
portfolio.individualPosition['Description'].unique()

array(['FIDELITY 500 INDEX FUND',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 09/11/2025',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 09/16/2025',
       'FIDELITY TOTAL MARKET INDEX FUND',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 08/07/2025',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 08/26/2025',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 09/09/2025',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 09/30/2025',
       'FIDELITY INTERNATL INDEX FUND',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 10/30/2025',
       'HELD IN MONEY MARKET', 'MICROSOFT CORP', 'APPLE INC',
       'TESLA INC COM', 'MCDONALD S CORP',
       'BERKSHIRE HATHAWAY INC COM USD0.0033 CLASS B',
       'ALPHABET INC CAP STK CL A', 'AMERICAN EXPRESS CO COM USD0.20',
       'JPMORGAN CHASE &CO. COM', 'KRAFT HEINZ CO COM',
       'STARBUCKS CORP COM USD0.001', 'COCA COLA CONS INC COM',
       'NIKE INC CLASS B COM NPV', 'VERIZON COMMUNICATIONS INC',
       'AMAZ

In [69]:
pprint(portfolio.individualTransactions['Description'].unique())

array(['No Description', 'FIDELITY TREASURY MONEY MARKET FUND',
       'APPLE INC', 'STARBUCKS CORP COM USD0.001',
       'JPMORGAN CHASE &CO. COM', 'AMERICAN EXPRESS CO COM USD0.20',
       'ALPHABET INC CAP STK CL A', 'AMAZON.COM INC',
       'NIKE INC CLASS B COM NPV',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 03/26/2024',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 03/21/2024',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 02/15/2024',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 04/04/2024',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 04/25/2024',
       'TESLA INC COM', 'FIDELITY 500 INDEX FUND',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/09/2024',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 06/06/2024',
       'MCDONALD S CORP',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 07/11/2024',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/30/2024',
       'UNITED STATES TREAS BILLS ZERO CPN 0.00000% 10/24/2024',


In [75]:
portfolio.individualTransactions[portfolio.individualTransactions['Description']=='FIDELITY TREASURY MONEY MARKET FUND']

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
1,2022-07-29,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,0.22,NaT
2,2022-07-29,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.22,1.0,NaN,NaN,NaN,-0.22,NaT
15,2022-08-31,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,32.21,NaT
16,2022-08-31,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,32.21,1.0,NaN,NaN,NaN,-32.21,NaT
19,2022-09-30,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,34.02,1.0,NaN,NaN,NaN,-34.02,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,2025-05-30,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET FU...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,550.06,1.0,NaN,NaN,NaN,-550.06,NaT
556,2025-06-30,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET FU...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,491.39,1.0,NaN,NaN,NaN,-491.39,NaT
559,2025-06-30,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MARK...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,491.39,NaT
592,2025-07-31,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MARK...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,111.12,NaT
